In [1]:
import pathlib
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import anndata
from pybedtools import BedTool

## Parameter

In [2]:
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial/ITSpatial_rms_results_collapsed.tsv'
dmr_prefix = 'DGmCH'

black_list_path = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'
skip_quantile = 0.25
delta_to_mean = 0.1
dms_cutoff = 1

In [3]:
# Parameters
dmr_path = "/home/hanliu/project/mouse_rostral_brain/DMR/SubType/raw/SubType-chrX/SubType-chrX_rms_results_collapsed.tsv"
dmr_prefix = "Sub"
black_list_path = "/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz"
skip_quantile = 0.25
delta_to_mean = 0.3
dms_cutoff = 1


## Read DMR

In [4]:
# get column and n_sample
dmr_column = pd.read_csv(dmr_path, sep='\t', nrows=0).columns
n_sample = dmr_column.size - 6
dmr_column = dmr_column.map(lambda i: i[18:] if 'methylation_level_' in i else i)
samples = dmr_column[6:]

dmr_region_infos = []
mc_rates = []
hypo_sig_dict = {sample: [] for sample in samples}
hyper_sig_dict = {sample: [] for sample in samples}

dmr_df = pd.read_csv(dmr_path, sep='\t', skiprows=1, names=dmr_column)
print(dmr_df.shape[0], 'dmr before filter')
dmr_df.index = dmr_df.index.map(lambda i: f'{dmr_prefix}_{i}')

# save raw info
dmr_bed = dmr_df.iloc[:, :4].copy()
mc_rate = dmr_df.iloc[:, 6:].copy()

187296 dmr before filter


## Save unfiltered dmr info

In [5]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    hdf['Rate'] = mc_rate
    hdf['Bed'] = dmr_bed

In [6]:
dmr_bed.reset_index().iloc[:, [1, 2, 3, 0]].to_csv(pathlib.Path(dmr_path).parent / 'TotalDMR.nofilter.bed', 
                                                   index=None, header=None, sep='\t')

## Filter DMR based on min-max delta and blacklist

In [7]:
# filter by min max delta first, if DMR do not pass min max filter, it won't have sig sample anyway
delta_judge = (mc_rate.max(axis=1) - mc_rate.min(axis=1)) > delta_to_mean
delta_index = mc_rate.index[delta_judge]

print(delta_index.size, 'dmr pass delta filter')

184803 dmr pass delta filter


In [8]:
# filter by blacklist
black_list = pd.read_csv(black_list_path, sep='\t', header=None)
black_list = BedTool.from_dataframe(black_list)

_dmr_bed = BedTool.from_dataframe(dmr_bed.reset_index().iloc[:, [1,2,3,0]])
filtered_dmr_bed = _dmr_bed.intersect(black_list, v=True).to_dataframe().set_index('name')

white_index = filtered_dmr_bed.index
print(white_index.size, 'dmr pass blacklist filter')

184097 dmr pass blacklist filter


In [9]:
dmr_df = dmr_df.loc[delta_index & white_index]
dmr_df = dmr_df[dmr_df['number_of_dms'] >= dms_cutoff]
print(dmr_df.shape[0], 'dmr after filter')
dmr_df.head()

181675 dmr after filter


,#chr,start,end,number_of_dms,hypermethylated_samples,hypomethylated_samples,CT-L6_Il1rap,NP-L6_Cntnap5a,CGE-Lamp5_Sorcs1,CGE-Vip_Grm8,...,Foxp2_Trpc7,CGE-Lamp5_Grk5,IT-L5_Cdh8,IG-CA2_Peak1,ANP_anp-dg,OLF_Xkr6,D1L-Fstl4_Cadm1,IT-L4_Shc3,D1L-PAL_Plcxd3,L6b_Pkhd1
Sub_0,chrX,3052540,3052540,1,NaN,NaN,0.761905,0.888889,0.875,0.666667,...,0.777778,0.812500,0.869565,NaN,0.777778,NaN,0.750000,1.000000,0.526316,NaN
Sub_1,chrX,3055605,3055645,2,NaN,"ODC_odc-small,VLMC_Mapk4,ODC_odc-large,PC_pc-a...",0.886792,0.933333,0.800,1.000000,...,0.844444,0.821429,0.877551,0.733333,1.000000,0.625,0.555556,0.929825,0.937500,NaN
Sub_2,chrX,3057142,3057142,1,NaN,"CGE-Vip_Ntng1,MGC_mgc-all,ODC_odc-small,MGE-Pv...",0.730769,0.647059,0.600,1.000000,...,0.578947,0.750000,0.727273,1.000000,NaN,NaN,0.500000,0.619048,0.700000,0.000000
Sub_3,chrX,3057776,3057980,3,NaN,"ASC_cortex-olf,MGE-Sst_Bmper,VLMC_Mapk4,IT-L4_...",0.854305,0.872727,0.880,0.812500,...,0.864407,0.779412,0.886667,0.837838,0.954545,0.875,0.918367,0.878453,0.806452,0.857143
Sub_4,chrX,3058238,3058238,1,"MGE-Sst_Bmper,PT-L5_Tenm2,L6b_Nrp2","MGC_mgc-all,PAL-Inh_Tcf7l2,ODC_odc-small,Foxp2...",0.826087,0.642857,1.000,0.000000,...,0.700000,0.782609,0.666667,0.714286,0.666667,NaN,0.500000,0.700000,0.571429,0.500000


## Calculate sample delta to robust mean

In [10]:
mc_rate = mc_rate.loc[dmr_df.index].copy()
n_sample = mc_rate.shape[1]

assert skip_quantile < 0.5
skip_n = min(1, round(n_sample * skip_quantile))

In [11]:
# ~1M rows / min, depending on n_sample, not very slow
def robust_mean(row):
    return np.nanmean(np.sort(row)[skip_n:-skip_n])
robust_mean = np.apply_along_axis(robust_mean, 1, mc_rate.values)
robust_mean = pd.Series(robust_mean, index=mc_rate.index)

In [12]:
delta = mc_rate - robust_mean[:, None]
# 0 is hypo DMR, 1 is hyper DMR, nan is not significant
judge = (delta > 0)[delta.abs() > delta_to_mean]

In [13]:
robust_mean_hypo_records = {}
robust_mean_hyper_records = {}

for sample, sample_col in judge.iteritems():
    hypo_dmr = sample_col[sample_col == 0].index
    hyper_dmr = sample_col[sample_col == 1].index
    
    robust_mean_hypo_records[sample] = hypo_dmr
    robust_mean_hyper_records[sample] = hyper_dmr

## Assign DMR to each sample

In [14]:
# parse results and add the robust mean judge
for sample in samples:
    # here saved a list of np.array
    hypo_index = dmr_df[dmr_df['hypomethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hyper_index = dmr_df[dmr_df['hypermethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hypo_sig_dict[sample] = hypo_index & robust_mean_hypo_records[sample]
    hyper_sig_dict[sample] = hyper_index & robust_mean_hyper_records[sample]
    print(sample, 'HypoDMR ', hypo_sig_dict[sample].size, sep='\t')
    print(sample, 'HyperDMR', hyper_sig_dict[sample].size, sep='\t')


CT-L6_Il1rap	HypoDMR 	7088
CT-L6_Il1rap	HyperDMR	1048
NP-L6_Cntnap5a	HypoDMR 	2231
NP-L6_Cntnap5a	HyperDMR	4066


CGE-Lamp5_Sorcs1	HypoDMR 	7368
CGE-Lamp5_Sorcs1	HyperDMR	1287
CGE-Vip_Grm8	HypoDMR 	5039
CGE-Vip_Grm8	HyperDMR	1108


LSX-Inh_Dock10	HypoDMR 	3258
LSX-Inh_Dock10	HyperDMR	3906
CGE-Vip_Ccser1	HypoDMR 	5092
CGE-Vip_Ccser1	HyperDMR	1982


CGE-Vip_Ntng1	HypoDMR 	3775
CGE-Vip_Ntng1	HyperDMR	534
ASC_cortex-olf	HypoDMR 	15695
ASC_cortex-olf	HyperDMR	3381


MGE-Sst_Chodl	HypoDMR 	3824
MGE-Sst_Chodl	HyperDMR	1188
IT-L6_Oxr1	HypoDMR 	6198
IT-L6_Oxr1	HyperDMR	646


...

Foxp2_Trpc7	HypoDMR 	7659
Foxp2_Trpc7	HyperDMR	2072
CGE-Lamp5_Grk5	HypoDMR 	12764
CGE-Lamp5_Grk5	HyperDMR	1332


IT-L5_Cdh8	HypoDMR 	13830
IT-L5_Cdh8	HyperDMR	497
IG-CA2_Peak1	HypoDMR 	14169
IG-CA2_Peak1	HyperDMR	969


ANP_anp-dg	HypoDMR 	17383
ANP_anp-dg	HyperDMR	1011
OLF_Xkr6	HypoDMR 	6038
OLF_Xkr6	HyperDMR	429


D1L-Fstl4_Cadm1	HypoDMR 	6884
D1L-Fstl4_Cadm1	HyperDMR	1763
IT-L4_Shc3	HypoDMR 	11225
IT-L4_Shc3	HyperDMR	633


D1L-PAL_Plcxd3	HypoDMR 	7087
D1L-PAL_Plcxd3	HyperDMR	3257
L6b_Pkhd1	HypoDMR 	3048
L6b_Pkhd1	HyperDMR	147


In [15]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    for sample, hypo_index in hypo_sig_dict.items():
        hdf[f'HypoDMR/{sample}'] = pd.Series(hypo_index)
    for sample, hyper_index in hyper_sig_dict.items():
        hdf[f'HyperDMR/{sample}'] = pd.Series(hyper_index)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'CT-L6_Il1rap'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'NP-L6_Cntnap5a'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'CGE-Lamp5_Sorcs1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to us

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'LSX-Inh_Zeb2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'DG-po_Bcl11a'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'PAL-Inh_Tmem178'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use n

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'MSN-D2_Casz1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L5_Etv1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'PT-L5_Nectin1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natur

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'LSX-Inh_Cacna1i'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'NP-L6_Olfml2b'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'MGE-Pvalb_Sema5a'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to 

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'ANP_anp-dg'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'D1L-Fstl4_Cadm1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L4_Shc3'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natur

## Dump DMR bed

In [16]:
hypo_dir = pathlib.Path(dmr_path).parent / 'HypoDMR'
hypo_dir.mkdir(exist_ok=True)
for sample, hypo_index in hypo_sig_dict.items():
    _bed = dmr_bed.loc[hypo_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hypo_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

In [17]:
hyper_dir = pathlib.Path(dmr_path).parent / 'HyperDMR'
hyper_dir.mkdir(exist_ok=True)
for sample, hyper_index in hyper_sig_dict.items():
    _bed = dmr_bed.loc[hyper_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hyper_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

## DMR hits matrix

In [18]:
sig_dict = hypo_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hypo_dir / 'TotalHits.h5ad')


... storing '#chr' as categorical


In [19]:
sig_dict = hyper_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hyper_dir / 'TotalHits.h5ad')

... storing '#chr' as categorical
